# Combinação e rejeição de classificadores

In [1]:
import pandas as pd
import numpy as np
import pickle
import os

In [2]:
resultados = pd.read_csv('resultados.csv')
resultados

,Árvore,RandomForest,KNN,Regressão Logistica,SVM,Redes Neurais
0,0.9865,0.9800,0.9800,0.9475,0.9835,0.9970
1,0.9845,0.9850,0.9800,0.9465,0.9840,0.9970
2,0.9905,0.9855,0.9800,0.9470,0.9835,0.9970
3,0.9875,0.9835,0.9800,0.9460,0.9845,0.9970
4,0.9885,0.9860,0.9810,0.9465,0.9855,0.9975
5,0.9885,0.9820,0.9785,0.9465,0.9835,0.9970
6,0.9885,0.9850,0.9800,0.9470,0.9840,0.9970
7,0.9875,0.9845,0.9805,0.9480,0.9845,0.9970
8,0.9860,0.9825,0.9810,0.9465,0.9820,0.9970
9,0.9875,0.9845,0.9810,0.9465,0.9830,0.9970


Melhores resultados:
- Rede Neural
- Árvore de decisão
- SVM

In [3]:
rede_neural = pickle.load(open('rede_neural_finalizada.sav','rb'))
arvore = pickle.load(open('arvore_finalizada.sav','rb'))
svm = pickle.load(open('svm_finalizada.sav','rb'))

In [4]:
with open('credit.pkl', 'rb') as f:
    X_credit_train, y_credit_train, X_credit_test, y_credit_test = pickle.load(f)
    
X_credit = np.concatenate((X_credit_train, X_credit_test), axis=0)
y_credit = np.concatenate((y_credit_train, y_credit_test), axis=0) 

In [5]:
print(X_credit.shape)
print(y_credit.shape)

(2000, 3)
(2000,)


## Combinação de classificadores

In [6]:
novo_registro = X_credit[0].reshape(1,-1)
print(novo_registro.shape)
novo_registro

(1, 3)


array([[-1.3754462 ,  0.50630973,  0.10980934]])

In [7]:
resposta_rede = rede_neural.predict(novo_registro)
resposta_arvore = arvore.predict(novo_registro)
resposta_svm = svm.predict(novo_registro)

In [8]:
respostas = resposta_rede[0], resposta_arvore[0], resposta_svm[0]
respostas 

(0, 0, 0)

Os três algoritmos apresentaram o mesmo resultado.

In [9]:
paga = 0
n_paga = 0
for i in respostas:
    if i == 1:
        n_paga += 1
    else:
        paga +=1

if paga > n_paga:
    print('Cliente pagará o empréstimo')
elif paga == n_paga:
    print('Empate')
else:
    print('Cliente não pagará o empréstimo')

Cliente pagará o empréstimo


Poderíamos ainda colocar peso para cada algoritmo de acordo com sua qualidade

## Rejeição de classificadores

In [10]:
probabilidade_rede = rede_neural.predict_proba(novo_registro)
probabilidade_arvore = arvore.predict_proba(novo_registro)
probabilidade_svm = svm.predict_proba(novo_registro)

In [11]:
probabilidade_rede, probabilidade_arvore, probabilidade_svm

(array([[1.00000000e+00, 3.99105617e-17]]),
 array([[1., 0.]]),
 array([[9.99998418e-01, 1.58188555e-06]]))

In [12]:
confianca_rede = probabilidade_rede.max()
confianca_arvore = probabilidade_arvore.max()
confianca_svm = probabilidade_svm.max()

In [13]:
confiancas = confianca_rede, confianca_arvore, confianca_svm
confiancas

(1.0, 1.0, 0.9999984181144514)

In [14]:
#preciso otimizar essa função!!!!
paga = 0
nao_paga = 0
confianca_minima = 0.999999
algoritmos = 0

if confianca_rede >= confianca_minima:
  algoritmos += 1
  if resposta_rede[0] == 1:
    nao_paga += 1
  else:
    paga += 1

if confianca_arvore >= confianca_minima:
  algoritmos += 1
  if resposta_arvore[0] == 1:
    nao_paga += 1
  else:
    paga += 1

if confianca_svm >= confianca_minima:
  algoritmos += 1
  if resposta_svm[0] == 1:
    nao_paga += 1
  else:
    paga += 1

if paga > nao_paga:
  print('Cliente pagará o empréstimo, baseado em {} algoritmos'.format(algoritmos))
elif paga == nao_paga:
  print('Empate, baseado em {} algoritmos'.format(algoritmos))
else:
  print('Cliente não pagará o empréstimo, baseado em {} algoritmos'.format(algoritmos))

Cliente pagará o empréstimo, baseado em 2 algoritmos
